In [ ]:
# PASSO 1: INSTALAR AS BIBLIOTECAS
!pip install -q langchain langchain-groq langchainhub

print("Bibliotecas instaladas!")

# PASSO 2: IMPORTAR TUDO E CONFIGURAR A API KEY

import os
import getpass
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent, tool
from langchain import hub

# Pedir a chave de API de forma segura
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Cole sua Chave de API da Groq aqui: ")

print(" Chave de API da Groq configurada")

# PASSO 3: DEFINIR A FERRAMENTA
@tool
def calculadora(expression: str) -> str:
    """
    Use esta ferramenta para resolver qualquer operação matemática.
    A entrada deve ser uma string válida que possa ser calculada, como '2+2' ou '(10*5)/2'.
    """
    try:
        result = eval(expression, {'__builtins__': None}, {})
        return str(result)
    except Exception as e:
        return f"Erro ao calcular a expressão: {e}"

tools = [calculadora]
print("✅ Ferramenta 'calculadora' criada com sucesso!")

# PASSO 4: CRIAR O AGENTE COM O MODELO MAIS PESADO DA GALÁXIA PQ É UMA API, SE USAR EM PC PESSOAL CONSIDERAR MODELO MENORES

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile"
)

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print(" Agente  criado com o Llama 3.3 70b (VERIFICADO!) e pronto!")

# PASSO 5: EXECUTAR

print("\n--- INICIANDO TESTES ---")

try:
    print("\nPERGUNTA: Quanto é 125 mais 345?")
    response1 = agent_executor.invoke({"input": "Quanto é 125 mais 345?"})
    print("\nRESPOSTA FINAL:", response1['output'])

    print("\n\nPERGUNTA: Qual o resultado de (1024 / 16) * 3 - 50?")
    response2 = agent_executor.invoke({"input": "Qual o resultado de (1024 / 16) * 3 - 50?"})
    print("\nRESPOSTA FINAL:", response2['output'])

    print("\n\nPERGUNTA: Quem descobriu o Brasil?")
    response3 = agent_executor.invoke({"input": "Quem descobriu o Brasil?"})
    print("\nRESPOSTA FINAL:", response3['output'])

except Exception as e:
    print(f"\n Ocorreu um erro durante a execução: {e}")

print("\n--- TESTES FINALIZADOS ---")